In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
cast=pd.read_csv('/home/featurize/data/40000/credits.csv')
df2=pd.read_csv('/home/featurize/data/40000/movies_metadata.csv')
New = pd.read_csv('/home/featurize/data/40000/New.csv')
cast['id']=cast['id'].astype(str)
df2['id']=df2['id'].astype(str)

/home/jovyan/.pyenv/versions/3.7.4/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
#join the data on id
new = pd.merge(cast,df2,on = "id",how = "left")

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
new['overview'] = new['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(new['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape #over 75827 words to describe 45538 movies

(45538, 75827)

In [22]:
#Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
#cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#Construct a reverse map to identify the index of a movie in our metadata DataFrame, given its title
indices = pd.Series(new.index, index=new['title']).drop_duplicates()

18288

In [12]:
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return new['title'].iloc[movie_indices]

In [8]:
get_recommendations('The Dark Knight Rises')

12501                                      The Dark Knight
150                                         Batman Forever
1331                                        Batman Returns
15541                           Batman: Under the Red Hood
585                                                 Batman
21235    Batman Unmasked: The Psychology of the Dark Kn...
9244                    Batman Beyond: Return of the Joker
18071                                     Batman: Year One
19827              Batman: The Dark Knight Returns, Part 1
3100                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [3]:
New = pd.read_csv('/home/featurize/data/40000/New.csv')

# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(New['combine'])

# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

# Reset index of our main DataFrame and construct reverse mapping as before
indices = pd.Series(New.index, index=New['title'])

In [5]:
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return New['title'].iloc[movie_indices]

get_recommendations('The Dark Knight Rises', cosine_sim2)

12490    The Dark Knight
10129      Batman Begins
11364       The Prestige
2467           Following
44677            Dunkirk
5259            Insomnia
22880       Interstellar
15489          Inception
25898          Doodlebug
9236              Shiner
Name: title, dtype: object